This file downloads satellite imagery using Sentinel API

In [14]:
#!pip install sentinelsat

In [1]:
# imports
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

import os
from urllib import request
from datetime import date
import datetime
import pandas as pd
import json
import zipfile

In [16]:
# constants
CWD = os.getcwd()
URL = 'https://developers.google.com/public-data/docs/canonical/countries_csv'
COU_DIR = '/data/country/'
IMG_DIR = '/data/image/'
USER = 'connorhynes'
PASS = 'or8pitJ4N#slSq&am87q^WsfA'
DATE_END = date.today()

# adjustable search parameters
COUNTRY = 'FR' # abbreviation of country
YEAR_SEARCH = 1 # how many previous years to query
CLOUD_COVER = [0, 10] # acceptable amount of cloud cover [start, end] (percentage)

In [13]:
# requests country data from google, returns dataframe of country data
def getCountry(URL):
    webpage = request.urlopen(URL) # why can i query a webpage and suddenly extract just the table
    country_data = pd.read_html(webpage)[0] # i think the 0 specified data frame (type)? 
    return country_data


In [4]:
# get desired country from user input, returns lat/long and country name
def userInput(URL, countries):
    #getCountry(URL)
    country_ab = COUNTRY
    row = countries[countries.country == country_ab]
    lat, long, cou_name = row.latitude.values[0], row.longitude.values[0], row.country.values[0]
    return(lat, long, cou_name)


In [5]:
# creates geojson from desired lat/long, used for footprint of API query
def createGeoJson(URL, countries):
    # get country information
    lat, long, cou_name = userInput(URL, countries)
    
    # construct geojson with lat/long
    data = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point", 
                    "coordinates": [long, lat]
                }
            }
        ]
    }
    
    # filepath where .geojson will be saved
    filepath = CWD + COU_DIR + cou_name + '.geojson'
    
    # writes geojson file to directory
    with open(filepath, "w", encoding = 'utf-8') as f:
        json.dump(data, f, ensure_ascii = False, indent = 4) # formatting
    f.close()
    return filepath
    

In [6]:
# determines how many years to query based on YEAR_SEARCH variable, returns date 
def getStartDate():
    DATE_START = DATE_END - pd.DateOffset(years = YEAR_SEARCH)
    return DATE_START


In [7]:
# connect to API service, requests based on search parameters, returns sorted dataframe of products
def sentinel_product(filepath, api):
    footprint = geojson_to_wkt(read_geojson(filepath))

    products = api.query(footprint,
                         date = (getStartDate(), DATE_END),
                         platformname = 'Sentinel-2',
                         cloudcoverpercentage = CLOUD_COVER)


    products_df = api.to_dataframe(products).sort_values(by = ['summary'], ascending = False)

    return products_df


In [8]:
def sentinel_contact():
    api = SentinelAPI(USER, PASS, 'https://apihub.copernicus.eu/apihub')
    return api

In [9]:
# uses product dataframe parameter to download first satellite imagery, returns file title
def sentinel_download(products_df, api):

    title = products_df.iloc[0]['title']
    prod_id = products_df.iloc[0]['uuid']

    api.download(prod_id, CWD + IMG_DIR)
    
    return title


In [10]:
# unpacks downloaded file using title parameter, removes zipped version
def unpack(title):
    dl_filepath = CWD + IMG_DIR + title + '.zip'
    with zipfile.ZipFile(dl_filepath, 'r') as zip_ref:
        zip_ref.extractall(CWD + IMG_DIR)
    os.remove(dl_filepath)
    

In [17]:
def main():
    countries = getCountry(URL)
    filepath = createGeoJson(URL, countries)
    api = sentinel_contact()
    products_df = sentinel_product(filepath, api)
    title = sentinel_download(products_df, api)
    unpack(title)
    print("Complete.")
    

In [18]:
main()


MD5 checksumming:   0%|          | 0.00/825M [00:00<?, ?B/s]

Complete.
